In [5]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from pprint import pprint as pp

In [ ]:
# First run ./update_pickles.sh if needed

# Choosing a set of dataframes
pickle_name = 'c19_all'
# Choosing a data type
df_type = 'cases'

pickle_file = open(f'../pickles/{pickle_name}''.p', 'rb')
df= pickle.load(pickle_file)[df_type]
pickle_file.close()